# 数独 Solver


In [1]:
import copy
import re
import numpy as np
from colorama import Fore

In [198]:
class Sudoku(object):
    def __init__(self):
        self.cell = {}
        self.solved = None

    def load_game(self, fn):
        with open(fn, 'r') as f:
            for line in f:
                line = line.strip()
                if line == "":
                    continue
                m = re.match('(\d)x(\d)\s*(\d)', line)
                if m is None:
                    continue
                key = m.group(1) + "x" + m.group(2)
                self.cell[key] = int(m.group(3))
        self.initial_cell = copy.copy(self.cell)

    def get_cell(self, x, y):
        key = f'{x}x{y}'
        if key in self.cell:
            return self.cell[key]
        else:
            return 0

    def draw(self):
        for y in range(9):
            if y % 3 == 0:
                print(Fore.WHITE + "------------")
            for x in range(9):
                if x % 3 == 0:
                    print(Fore.WHITE + "|", end="")
                key = f'{x}x{y}'
                if key in self.cell:
                    color = Fore.RED
                    if key in self.initial_cell:
                        color = Fore.WHITE

                    v = color + str(self.cell[key])
                else:
                    v = Fore.WHITE + " "
                print(v, end="")

            print()
        print()
        
    # セルを全て舐めて、正解が１個のセルを決定する
    def step1(self):
        num_solved = 0
        for x in range(9):
            for y in range(9):
                key = f'{x}x{y}'
                c = self.peek(x, y)

                if c is None:
                    continue
                if len(c) == 1:
                    self.cell[key] = c[0]
                    num_solved += 1
                elif len(c) == 0:
                    print(f'step1 {key} no entry')
                    return -1
        return num_solved
    
    # 空きのセルに候補の数字のリストを設定
    def step2(self):
        remain = {}
        for x in range(9):
            for y in range(9):
                key = f'{x}x{y}'
                c = self.peek(x, y)
                if c is None:
                    continue
                if len(c) > 1:
                    #self.cell[key] = c
                    #print(f'{key} {c}')
                    remain[key] = c

        return remain

    def issolved(self):
        if self.solved:
            return True
        for x in range(9):
            for y in range(9):
                key = f'{x}x{y}'
                if key not in self.cell:
                    return False
        self.solved = True
        return True
    
    # 指定セルの設置可能数値のリストを取得
    def peek(self, x, y):
        key = f'{x}x{y}'
        if key in self.cell:
            return None
        hlist = self.getHolizonValue(y)
        #print(f'h = {hlist}')
        vlist = self.getVerticalValue(x)
        #print(f'v = {vlist}')
        clist = self.get9CellValue(x, y)
        #print(f'c = {clist}')
        # 現在のセル＋縦リスト＋横リストの重複しないセットを取得
        other_list = set(hlist + vlist + clist)
        #print(f'peek {key} {other_list}')
        candidate = []
        for v in range(1, 10):
            if v not in other_list:
                candidate.append(v)

        return candidate
    
    # 未設定のセル数を取得
    def get_num_remain(self):
        num = 0
        for x in range(9):
            for y in range(9):
                key = f'{x}x{y}'
                if key in self.cell:
                    c = self.cell[key]
                    if type(c) is list and len(c) > 1:
                        num += 1
                else:
                    num += 1
        return num

    # 横方向の数値のリストを取得
    def getHolizonValue(self, y):
        values = []
        for x in range(9):
            key = f'{x}x{y}'
            if key in self.cell and type(self.cell[key]) is int:
                values.append(self.cell[key])

        return values;
    

    # 縦方向の数値リストを取得
    def getVerticalValue(self, x):
        values = []
        for y in range(9):
            key = f'{x}x{y}'
            if key in self.cell and type(self.cell[key]) is int:
                values.append(self.cell[key])

        return values;

    # 指定の９セル内の数値リストを取得
    def get9CellValue(self, x, y):
        values = []
        base_x = x // 3 * 3
        base_y = y // 3 * 3
        for x in range(3):
            for y in range(3):
                key = f'{base_x + x}x{base_y + y}'
                if key in self.cell and type(self.cell[key]) is int:
                    values.append(self.cell[key])
        return values;
    

    def solve_basic(self):
        nstep = 0
        while True:
            nstep += 1
            print(f'solve step {nstep}')
            n = self.step1()
            self.draw()
            if n < 0:
                return -1
            if n == 0:
                break
            #if n > 0:
            #    self.draw()
        return nstep
    
    def solve(self, deep=0):
        if self.issolved():
            return

        self.draw()
        if self.solve_basic() < 0:
            return
        if self.issolved():
            print("+++++++++ SOLVED ++++++++++++")
            return

        remain = self.step2()
        remain_sorted = sorted(remain.items(), key=lambda x:len(x[1]))
        if len(remain_sorted) == 0:
            return


        #print(f'remain {remain_sorted}')
        mark_cell = remain_sorted[0]
        for i in range(len(mark_cell[1])):
            copy_cell = copy.copy(self.cell)
            self.cell[mark_cell[0]] = mark_cell[1][i]
            solve3_ret = self.solve(deep + 1)
            self.cell = copy.copy(copy_cell) 

In [199]:
sudoku = Sudoku()
sudoku.load_game('sudoku_ex/ex3.txt')
sudoku.draw()


------------
|9 5|   | 7 
|  8|7 3|  1
|4  |   |   
------------
|389|1  | 6 
|   |   |   
|6  |34 | 1 
------------
| 3 |57 |6 4
|   | 6 |   
| 5 |   |8  



In [200]:
sudoku.solve()

------------
|9 5|   | 7 
|  8|7 3|  1
|4  |   |   
------------
|389|1  | 6 
|   |   |   
|6  |34 | 1 
------------
| 3 |57 |6 4
|   | 6 |   
| 5 |   |8  

solve step 1
------------
|9 5|   | 7 
|268|7 3|  1
|4  |   |   
------------
|389|1  | 6 
|   |   |   
|6  |34 | 1 
------------
| 3 |57 |6 4
|   | 6 |   
| 5 |   |8  

solve step 2
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|   |   |   
|627|34 | 1 
------------
| 3 |57 |6 4
|   | 6 |   
| 5 |   |8  

solve step 3
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
| 41|   |   
|627|34 | 1 
------------
| 32|57 |694
| 94| 6 |   
| 56|   |8  

solve step 4
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
| 32|57 |694
| 94| 6 |   
| 56|   |8  

solve step 5
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
| 32|57 |694
| 94| 6 |   
| 56|  

|894|261|   
|756|934|82 

------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|57 |694
| 94| 6 |   
| 56|   |8  

solve step 1
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
| 94| 6 |   
| 56|   |8  

solve step 2
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
| 94| 6 |   
| 56|   |8  

------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|194| 6 |   
| 56|   |8  

solve step 1
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|194| 6 |   
|756|   |8  

solve step 2
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|194| 

|832|571|694
|194|268|   
|756|9  |8  

solve step 1
step1 8x8 no entry
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|194|268|   
|756|934|82 

------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|194|86 |   
|756|   |8  

solve step 1
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|194|862|   
|756|   |8  

solve step 2
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|194|862|   
|756|   |8  

------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|194|862|   
|756|4  |8  

solve step 1
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 


|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|26 |   
|156|   |8  

solve step 1
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|268|   
|156|   |8  

solve step 2
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|268|   
|156|   |8  

------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|268|   
|156|4  |8  

solve step 1
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|268|   
|156|4 9|8  

solve step 2
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|345|918
------------
|832|571|694
|794|268|   
|156|439|827

solve step 3
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1

|389|1  | 6 
|541|8  | 3 
|627|34 | 1 
------------
|832|571|694
|794|268| 53
|156|934|827

solve step 2
------------
|915|482|376
|268|7 3| 41
|473|6  | 8 
------------
|389|1  | 6 
|541|8  | 3 
|627|34 | 1 
------------
|832|571|694
|794|268|153
|156|934|827

solve step 3
------------
|915|482|376
|268|7 3| 41
|473|6  | 8 
------------
|389|1  | 6 
|541|8  | 3 
|627|34 | 1 
------------
|832|571|694
|794|268|153
|156|934|827

------------
|915|482|376
|268|753| 41
|473|6  | 8 
------------
|389|1  | 6 
|541|8  | 3 
|627|34 | 1 
------------
|832|571|694
|794|268|153
|156|934|827

solve step 1
step1 6x5 no entry
------------
|915|482|376
|268|753|941
|473|6 9| 8 
------------
|389|12 | 6 
|541|89 | 3 
|627|345| 1 
------------
|832|571|694
|794|268|153
|156|934|827

------------
|915|482|376
|268|793| 41
|473|6  | 8 
------------
|389|1  | 6 
|541|8  | 3 
|627|34 | 1 
------------
|832|571|694
|794|268|153
|156|934|827

solve step 1
step1 6x5 no entry
------------
|915|482|376
|268|79

|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|862|   
|156|4  |8  

solve step 1
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|862|   
|156|4 9|8  

solve step 2
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|862|   
|156|439|827

solve step 3
------------
|915|   | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|862|   
|156|439|827

------------
|915|2  | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|862|   
|156|439|827

solve step 1
------------
|915|28 | 7 
|268|7 3|  1
|473|   |   
------------
|389|1  | 6 
|541|   |   
|627|34 | 1 
------------
|832|571|694
|794|862|   
|156|439|827

solve step 2
------------
|915|2

In [194]:
sudoku.issolved()

True